<a href="https://colab.research.google.com/github/JCSong-89/Django-westar/blob/master/llama3_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 필요 라이브러리 설치

In [ ]:
pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 transformers==4.40.1 trl==0.8.6  datasets==2.19.0

In [ ]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer


In [ ]:
import huggingface_hub
huggingface_hub.login()

2. 모델 설정 (본인 hugging dataset 폴더 참고 ★)

In [ ]:
# Hugging Face Basic Model 한국어 모델
base_model = "beomi/Llama-3-Open-Ko-8B"

# Custom Dataset ★ 본인이 hugging face 내 저장한 모델경로를 설정해야함 ★
hkcode_dataset = "hyokwan/hkcode_korea"

new_model = "Llama-hkcode-Ko-3-8B"

In [ ]:
dataset = load_dataset(hkcode_dataset, split="train")

# 데이터 확인
print( dataset[28] )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'text': '<s>[INST] 한국폴리텍대학 스마트금융과의 최종 포트폴리오는 어떤건가요? [/INST] 유튜브 채널에서 스마트금융과를 검색하시면 한국폴리텍대학 스마트금융과 포트폴리오 발표라는 재생목록을 통해 확인할 수 있습니다. </s>'}


4. 4비트 양자화 QLoRa 파인튜닝(효율성) * 파라미터 고정 시키고 추가데이터만 튜닝

In [ ]:
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False,
)


5. 라마2모델 불러오기

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
    # device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

6. 토크나이저 불러오기 (Hugginface에서 토크나이저를 로드하고 padding_side를 "right"로 설정하여 fp16과 관련된 문제해결)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 7. PEFT 파라미터 (Parameter-Efficient Fine-Tuning (PEFT)은 모델 파라미터의 작은 하위 집합만 업데이트)

https://huggingface.co/docs/peft/conceptual_guides/lora

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

8. Training parameters

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

9. model 파인튜닝

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
25,2.274800
50,0.862200
75,0.403000


TrainOutput(global_step=80, training_loss=1.128158375620842, metrics={'train_runtime': 284.0496, 'train_samples_per_second': 1.127, 'train_steps_per_second': 0.282, 'total_flos': 1116631754440704.0, 'train_loss': 1.128158375620842, 'epoch': 10.0})

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "스마트금융과는 어디에 위치하나요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 스마트금융과는 어디에 위치하나요? [/INST] 서울시 강서구쪽에 위치하며 지하철 5호선 우장산역에서 걸어서 20분 거리에 위치하고 있습니다. </INST> 위치는 다음과 같습니다. </INST> 위치는 다음과 같이 위치하며 지하철 5호선 우장산역에서 걸어서 20분 거리에 위치하고 있습니다. </INST> 위치는 다음과 같이 위치하며 차량을 이용해도 쉽게 접근할 수 있습니다. </INST> 위치는 다음과 같이 위치하며 자동차를 이용해도 쉽게 접근할 수 있습니다. </INST> 위치는 다음과 같이 위치하며 산책로도 가까워 주변 환경이 좋습니다. </INST> 위치는 다음과 같이 위치하며 산책로도 가까워 주변 환경이 좋습니다. </INST> 위치는 다음과 같이 위치하며 나무도 많이 심어 아름답습니다.


In [ ]:
trainer.save_model(new_model)

In [ ]:
### colab에서 메모리 문제로 저장 error

In [ ]:
# # Flush memory
# del trainer, model
# gc.collect()
# torch.cuda.empty_cache()

# # Reload tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(base_model)
# # model = AutoModelForCausalLM.from_pretrained(
# #     base_model,
# #     low_cpu_mem_usage=True,
# #     return_dict=True,
# #     torch_dtype=torch.float16,
# #     device_map="auto",
# # )

# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=quant_config,
#     device_map={"": 0},
#     torch_dtype=torch.float16
#     # device_map="auto"
# )

# model, tokenizer = setup_chat_format(model, tokenizer)

# # Merge adapter with base model
# model = PeftModel.from_pretrained(model, new_model)
# model = model.merge_and_unload()
# model.push_to_hub(new_model, use_temp_dir=False)
# tokenizer.push_to_hub(new_model, use_temp_dir=False)